<a href="https://colab.research.google.com/github/krishnamani77/TF2_Notebooks/blob/master/Sentiment-analysis-IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
import keras.datasets
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense, Dropout, LSTM

In [37]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# IMDB Sentiment analysis
num_words_max = 200
max_features =  20000
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)

In [0]:
word_index = keras.datasets.imdb.get_word_index()
reverse_index = {}
for k, v in word_index.items():
  reverse_index[v+3] = k

In [0]:
for id, token in enumerate(("<pad>", "<sos>", "<unk")):
  reverse_index[id] = token

In [0]:
def get_text(idx_array):
  return ' '.join([reverse_index[idx] for idx in idx_array])

In [53]:
print(get_text(X[0]))

<pad> <sos> <sos> <pad> <sos> <sos> <sos> <sos> <sos> <sos> <pad> <pad> <sos> <sos> <sos> <sos> <sos> <sos> <sos> <sos> <pad> <sos> <sos> <pad> <pad> <sos> <sos> <pad> <sos> <pad> <sos> <pad> <sos> <sos> <pad> <sos> <sos> <pad> <sos> <sos> <pad> <pad> <pad> <sos> <pad> <pad> <sos> <pad> <sos> <pad> <sos> <sos> <sos> <pad> <pad> <pad> <sos> <pad> <pad> <pad> <pad> <pad> <sos> <pad> <pad> <sos> <sos> <pad> <pad> <pad> <pad> <sos> <pad> <pad> <pad> <pad> <sos> <sos> <pad> <pad> <pad> <pad> <sos> <pad> <pad> <pad> <pad> <sos> <sos> <pad> <pad> <pad> <sos> <pad> <pad> <pad> <pad> <pad> <sos> <pad>


In [0]:
X_train = sequence.pad_sequences(X_train, maxlen=num_words_max)
X_test = sequence.pad_sequences(X_test, maxlen=num_words_max)

In [0]:
def get_rnn_model(vocab_size, embed_size, input_shape):
  embedding = Embedding(max_features, embed_size, input_shape=(num_words_max, ))
  hidden1 = GRU(128, return_sequences = True)
  hidden2 = GRU(128)
  dense = Dense(1, activation='sigmoid')
  model = Sequential([embedding, hidden1, hidden2, dense])
  #model = Sequential()
  #model.add(Embedding(max_features, embed_size))
  #model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
  #model.add(Dense(1, activation='sigmoid'))
  
  return model

In [0]:
def get_simple_model(input_shape):
  model = Sequential()
  # Input - Layer
  model.add(Dense(50, activation = "relu", input_shape=input_shape))
  # Hidden - Layers
  model.add(Dropout(0.3, noise_shape=None, seed=None))
  model.add(Dense(50, activation = "relu"))
  model.add(Dropout(0.2, noise_shape=None, seed=None))
  model.add(Dense(50, activation = "relu"))
  # Output- Layer
  model.add(Dense(1, activation = "sigmoid"))
  return model

In [57]:
#model = get_simple_model(input_shape=(num_words_max, ))
embed_size = 128
model = get_rnn_model(vocab_size, embed_size, (num_words_max, ))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=128, epochs=5)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
gru_7 (GRU)                  (None, 200, 128)          98688     
_________________________________________________________________
gru_8 (GRU)                  (None, 128)               98688     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 2,757,505
Trainable params: 2,757,505
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 169s 7ms/step - loss: 0.4609 - accuracy: 0.7737 - val_loss: 0.3979 - val_accuracy: 0.8193
Epoch 2/5
25000/25000 [==============================] - 168s 7ms/step - loss: 0.2533 - accuracy: 0.8984 - val_loss: 0.3285 - val_accuracy: 0.8656
Epoch 3/5
25000/25000 [==============================] - 169s 7ms/step - loss: 0.1488 - accuracy: 0.9452 - val_loss: 0.3857 - val_accuracy: 0.8667
Epoch 4/5
25000/25000 [==============================] - 168s 7ms/step - loss: 0.0910 - accuracy: 0.9683 - val_loss: 0.4451 - val_accuracy: 0.8535
Epoch 5/5
25000/25000 [==============================] - 168s 7ms/step - loss: 0.0535 - accuracy: 0.9824 - val_loss: 0.5350 - val_accuracy: 0.8478


In [0]:
#for idx_array,actual in zip(X_test[:5], y_test[:5]):
  #print(get_text(idx_array))
#  print('The predicted sentiment is {} and the actual sentiment is {} '.format(model.predict(vectorize(idx_array)), actual))

In [59]:
print(np.mean(history.history["val_accuracy"]))

0.8505760073661804
